In [ ]:
#pip install CHAID

In [ ]:
from CHAID import Tree
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Leitura dos dados
pedidos = pd.read_table("C:/_Ayu/0-Fia/_2020/TURMAS/EA_EAD_ago20/Python/Aula2/PedidoOnLine.txt")
pedidos.head()

In [ ]:
pedidos.dtypes

In [ ]:
pedidos.shape

In [ ]:
pedidos.describe().transpose()

In [ ]:
# Verificando quantidade de missings
pedidos.isnull().sum()

In [ ]:
# Como a variável explicativa também deve ser categórica, 
# vamos segmentar a Idade em quartil
pedidos['Idade_q'] = pd.qcut(pedidos.Idade, 4)

In [ ]:
# Tabela Bidimensional: covariável x resposta com normalize = 'index' calculando o percentual, sem esta parte, fica apenas o absoluto
Idade_tab = pd.crosstab(pedidos["Idade_q"],pedidos["Influenciado_por_nota"], normalize='index')
Idade_tab


In [ ]:
Genero_tab = pd.crosstab(pedidos["Genero"],pedidos["Influenciado_por_nota"], normalize='index')
Genero_tab

## Exercício: Construa a tabela para as demais variáveis

In [ ]:
Estado_civil_tab 

In [ ]:
Renda_mensal_tab 

In [ ]:
Refeicao_mais_frequente_tab 

In [ ]:
Avaliacao_media_anterior_tab 

## Modelo - Árvore de Decisão

In [ ]:
# Transformando a variável resposta em categórica
pedidos['resposta_cat'] = pedidos.Influenciado_por_nota.astype('category')

In [ ]:
var_explicativas = pedidos[[
    'Idade_q',
    'Genero', 
    'Estado_civil',
    'Renda_mensal',
    'Refeicao_mais_frequente',
    'Avaliacao_media_anterior']]

var_resposta = pedidos['resposta_cat']

In [ ]:
# Constrói o modelo de árvore
modelo = Tree.from_numpy(
    var_explicativas.to_numpy(), 
    var_resposta.to_numpy(), 
    split_titles=['Idade_q',
    'Genero', 
    'Estado_civil',
    'Renda_mensal',
    'Refeicao_mais_frequente',
    'Avaliacao_media_anterior'], 
    min_child_node_size=2)

In [ ]:
modelo.print_tree()

In [ ]:
modelo.classification_rules()

In [ ]:
# Salvando na base os nós
pedidos['node'] = modelo.node_predictions()
pedidos

In [ ]:
# Salvando a taxa de resposta por nó
probs = pedidos.groupby(['node']).agg({'Influenciado_por_nota':'mean'}).reset_index()
probs

In [ ]:
# Marcando se o nó é propenso
probs['propenso'] = np.where(probs['Influenciado_por_nota'] >= pedidos['Influenciado_por_nota'].mean(), 1, 0)

In [ ]:
probs = probs.rename(columns={"Influenciado_por_nota":"prob"})
probs

In [ ]:
pedidos = pedidos.merge(probs, how='left', on='node')
pedidos

In [ ]:
tabela_desempenho = pd.crosstab(pedidos['Influenciado_por_nota'],pedidos['propenso'])

In [ ]:
tabela_desempenho

In [ ]:
acuracia = (tabela_desempenho[0][0] + tabela_desempenho[1][1])/tabela_desempenho.sum().sum()
acuracia

In [ ]:
sensibilidade = (tabela_desempenho[1][1])/(tabela_desempenho[1][1] + tabela_desempenho[0][1])
sensibilidade

In [ ]:
#Calcule a Especificidade
especificidade 